<a href="https://colab.research.google.com/github/ARJUN108-verma/Elite_Tech_internship/blob/main/Speech_Recognition_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Speech Recognition System

In [2]:
pip install torch torchaudio transformers SpeechRecognition pydub tk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 139.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [4]:
!python speech_recognition_app.py


python3: can't open file '/content/speech_recognition_app.py': [Errno 2] No such file or directory


In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import speech_recognition as sr
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import os

In [6]:
class SpeechRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Speech Recognition System")
        self.root.geometry("600x400")

        # Initialize models (lazy loading)
        self.sr_recognizer = None
        self.wav2vec_processor = None
        self.wav2vec_model = None

        self.create_widgets()

    def create_widgets(self):
        # Title
        title_label = tk.Label(self.root, text="Speech-to-Text System", font=("Arial", 16, "bold"))
        title_label.pack(pady=10)

        # File selection
        file_frame = tk.Frame(self.root)
        file_frame.pack(pady=10)

        self.file_path = tk.StringVar()
        file_entry = tk.Entry(file_frame, textvariable=self.file_path, width=50)
        file_entry.pack(side=tk.LEFT, padx=5)

        browse_btn = tk.Button(file_frame, text="Browse", command=self.browse_file)
        browse_btn.pack(side=tk.LEFT)

        # Method selection
        method_frame = tk.Frame(self.root)
        method_frame.pack(pady=10)

        tk.Label(method_frame, text="Select Method:").pack(side=tk.LEFT)

        self.method_var = tk.StringVar(value="speechrecognition")
        sr_radio = tk.Radiobutton(
            method_frame, text="SpeechRecognition (Google API)",
            variable=self.method_var, value="speechrecognition"
        )
        sr_radio.pack(side=tk.LEFT, padx=5)

        wav2vec_radio = tk.Radiobutton(
            method_frame, text="wav2vec (Local Model)",
            variable=self.method_var, value="wav2vec"
        )
        wav2vec_radio.pack(side=tk.LEFT, padx=5)

        # Transcribe button
        transcribe_btn = tk.Button(
            self.root, text="Transcribe Audio",
            command=self.transcribe_audio, height=2, width=20
        )
        transcribe_btn.pack(pady=20)

        # Results
        result_frame = tk.Frame(self.root)
        result_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        tk.Label(result_frame, text="Transcription Result:", font=("Arial", 12)).pack()

        self.result_text = tk.Text(result_frame, height=10, wrap=tk.WORD)
        self.result_text.pack(fill=tk.BOTH, expand=True, padx=10)

        # Progress bar
        self.progress = ttk.Progressbar(
            self.root, orient=tk.HORIZONTAL,
            length=300, mode='determinate'
        )
        self.progress.pack(pady=5)

        # Status bar
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        status_bar = tk.Label(
            self.root, textvariable=self.status_var,
            bd=1, relief=tk.SUNKEN, anchor=tk.W
        )
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)

    def browse_file(self):
        filetypes = [
            ("Audio files", "*.wav *.aiff *.aif *.flac *.mp3"),
            ("All files", "*.*")
        ]
        filename = filedialog.askopenfilename(
            title="Select an audio file",
            filetypes=filetypes
        )
        if filename:
            self.file_path.set(filename)
            self.status_var.set(f"Selected: {os.path.basename(filename)}")

    def transcribe_audio(self):
        audio_file = self.file_path.get()
        if not audio_file:
            messagebox.showerror("Error", "Please select an audio file first")
            return

        method = self.method_var.get()
        self.result_text.delete(1.0, tk.END)
        self.progress['value'] = 0
        self.root.update()

        try:
            if method == "speechrecognition":
                self.status_var.set("Transcribing using SpeechRecognition...")
                self.progress['value'] = 30
                self.root.update()

                result = self.transcribe_with_speechrecognition(audio_file)

                self.progress['value'] = 100
                self.status_var.set("Transcription complete")
            else:
                self.status_var.set("Transcribing using wav2vec...")
                self.progress['value'] = 30
                self.root.update()

                result = self.transcribe_with_wav2vec(audio_file)

                self.progress['value'] = 100
                self.status_var.set("Transcription complete")

            self.result_text.insert(tk.END, result)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")
            self.status_var.set("Error during transcription")
            self.progress['value'] = 0

    def transcribe_with_speechrecognition(self, audio_file):
        """Transcribe using SpeechRecognition library (Google API)"""
        if self.sr_recognizer is None:
            self.sr_recognizer = sr.Recognizer()
            self.status_var.set("Initializing SpeechRecognition...")
            self.root.update()

        try:
            with sr.AudioFile(audio_file) as source:
                audio_data = self.sr_recognizer.record(source)

            self.status_var.set("Sending to Google Speech Recognition...")
            self.progress['value'] = 60
            self.root.update()

            text = self.sr_recognizer.recognize_google(audio_data)
            return text

        except sr.UnknownValueError:
            return "Google Speech Recognition could not understand the audio"
        except sr.RequestError as e:
            return f"Could not request results from Google Speech Recognition service; {e}"

    def transcribe_with_wav2vec(self, audio_file):
        """Transcribe using wav2vec 2.0 model"""
        if self.wav2vec_processor is None or self.wav2vec_model is None:
            self.status_var.set("Loading wav2vec model (first time may take a while)...")
            self.root.update()

            # Use processor instead of tokenizer for better handling
            self.wav2vec_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
            self.wav2vec_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

        # Load audio file
        waveform, sample_rate = torchaudio.load(audio_file)

        # Convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # Resample to 16kHz if needed
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(sample_rate, 16000)
            waveform = resampler(waveform)

        self.status_var.set("Processing audio with wav2vec...")
        self.progress['value'] = 60
        self.root.update()

        # Preprocess and predict
        inputs = self.wav2vec_processor(
            waveform.squeeze().numpy(),
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        )

        with torch.no_grad():
            logits = self.wav2vec_model(inputs.input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = self.wav2vec_processor.batch_decode(predicted_ids)[0]

        return transcription

In [14]:
import matplotlib
matplotlib.use('Agg')  # Set the backend to Agg before importing pyplot

In [ ]:
import tkinter as tk
from tkinter import messagebox
import os
import sys

def is_display_available():
    """Check if a display is available"""
    return 'DISPLAY' in os.environ or sys.platform == 'darwin' or sys.platform.startswith('win')

def run_headless():
    """Command-line interface for headless systems"""
    print("Running in headless mode")
    # Implement your command-line functionality here
    audio_file = input("Enter audio file path: ")
    method = input("Choose method (1: SpeechRecognition, 2: wav2vec): ")

    # Your transcription logic here
    print(f"Would process {audio_file} with method {method}")

if __name__ == "__main__":
    if is_display_available():
        root = tk.Tk()
        from speech_recognition_app import SpeechRecognitionApp
        app = SpeechRecognitionApp(root)
        root.mainloop()
    else:
        run_headless()

Running in headless mode
